# Query Expansion

0) Just some imports

In [1]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [2]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeIDFs(self, documents):
        self.idfs = [math.log(len(documents) / sum([x in y.tokens for y in documents]), 2) for x in self.terms]
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeCorM(self, documents):
        A = [[y.tokens.count(x) for y in documents] for x in self.terms]
        t = [math.sqrt(sum(x**2 for x in y)) for y in A]
        At = np.array([[y/xt  for y in x] for x, xt in zip(A, t)])
        At_trans = At.transpose()
        result = At.dot(At_trans)
        for i in range(result.shape[0]):
            result[i,i] = -1.0
        self.corM = result
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [3]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [4]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [5]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    def computeBOW_Representation(self):
        self.bow = [self.tokens.count(x) for x in self.tokens]
    
    ### TODO - complete this method; it should compute a TF representation
    def computeTF_Representation(self):
        self.tf = [x/max(self.bow) for x in self.bow]
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    def computeTF_IDF_Representation(self):
        if(self.dictionary.idfs != []):
            self.tf_idf = [x - self.dictionary.idfs[self.dictionary.terms.index(y)] for x, y in zip(self.tf, self.tokens)]
        
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]
documents[73].computeRepresentations()
print(documents[73].tokens)
print(documents[1].bow)
print(documents[1].tf)

['machine', 'algorithm', 'group', 'dipartimento', 'the', 'machine', 'learning', 'group', 'the', 'dipartimento', 'informatica', 'universita', 'torino', 'part', 'larger', 'research', 'group', 'artificial', 'intelligence', 'and', 'description', 'research', 'learning', 'firstorder', 'classification', 'rule', 'firstorder', 'concept', 'description', 'genetic', 'algorithm']
[]
[]


1.4) Compute IDFs here

In [6]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['working', 6.459431618637298], ['www', 6.459431618637298], ['york', 6.459431618637298], ['young', 6.459431618637298], ['zdm', 6.459431618637298]]
[['learning', 0.03316686393519938], ['machine', 0.03316686393519938], ['the', 0.6520766965796931], ['and', 1.067114195858537], ['description', 1.289506617194985]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [7]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow)

[1, 1, 4, 4, 1, 4, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 4, 4, 2, 1, 1, 1]


1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [8]:
### TODO 
def getSimilarity(v1, v2):
    return sum([x * y for x, y in zip(v1, v2)]) / math.sqrt(sum([x * x for x in v1]) * sum([x * x for x in v2]))

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [9]:
query = "machine learning"
#query = "academic research"
#query = "international conference"
#query = "international conference washington"

In [10]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.10808624163830019 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.0993993094612981 | DOC ID = 54
Machine Learning Group
 The Machine Learning Group. Department of Computer and System Sciences
 Stockholm University/KTH Electrum 230 S-164 40 Kista, Sweden ... 


RANK = 3 WITH SIMILARITY = 0.09451508848241055 | DOC ID = 20
Machine Learning
 Machine Learning, THE ... Machine learning refers to a system capable of
 the autonomous acquisition and integration of knowledge. This ... 


RANK = 4 WITH SIMILARITY = 0.09322639147916306 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 5 WITH SIMILARITY = 0.09287837833194217 | DOC ID = 34
Machine Learning
 Mac

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [11]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

[[-1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -1.          0.         ...  0.18898224  0.
   0.        ]
 [ 0.          0.         -1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.18898224  0.         ... -1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         -1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -1.        ]]


2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [12]:
query = "machine"
query = "algorithm"
query = "learning"
query = "conference"
query = "research"
query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    print("Suggestions")
    index = dictionary.terms.index(query)
    ranks = [[x, y] for x, y in zip(dictionary.terms, dictionary.corM[index])]
    ranks.sort(key=lambda x: -x[1])
    i = 0
    while(ranks[i][1] > 0.5):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | WORD = " + str(ranks[i][0]))
        print("")
        i = i + 1
        
suggestKeywords(query, dictionary)

Suggestions
RANK = 1 WITH SIMILARITY = 0.5773502691896258 | WORD = deals

RANK = 2 WITH SIMILARITY = 0.5773502691896258 | WORD = dipartimento

RANK = 3 WITH SIMILARITY = 0.5773502691896258 | WORD = example

RANK = 4 WITH SIMILARITY = 0.5773502691896258 | WORD = firstorder

RANK = 5 WITH SIMILARITY = 0.5773502691896258 | WORD = following

RANK = 6 WITH SIMILARITY = 0.5773502691896258 | WORD = formal

RANK = 7 WITH SIMILARITY = 0.5773502691896258 | WORD = higherorder

RANK = 8 WITH SIMILARITY = 0.5773502691896258 | WORD = informatica

RANK = 9 WITH SIMILARITY = 0.5773502691896258 | WORD = part

RANK = 10 WITH SIMILARITY = 0.5773502691896258 | WORD = torino

RANK = 11 WITH SIMILARITY = 0.5773502691896258 | WORD = universita



# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [23]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    beta_part = sum([sum(documents[x].tf_idf) for x in rel_docs]) / len(rel_docs)
    gamma_part = sum([sum(documents[x].tf_idf) for x in nrel_docs]) / len(nrel_docs)
    qm = [alpha * q + (beta * beta_part) - (gamma * gamma_part) for q in qd.tf_idf]
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qm)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

RANK = 1 WITH SIMILARITY = 0.689265167532956 | DOC ID = 45
University of York Machine Learning Group
 Content Introduction. 


RANK = 2 WITH SIMILARITY = 0.6001401689675975 | DOC ID = 74
ECML/PKDD-2002
 ... Photos Proposals 2003 Sitemap Home ECML/PKDD-2002. 13th European Conference
 on Machine Learning (ECML'02). 6th European Conference on ... 


RANK = 3 WITH SIMILARITY = 0.5389659003750058 | DOC ID = 82
IJCAI 99 Workshop: Machine Learning for Information Filtering. Workshop
 on "Machine Learning for Information Filtering" at IJCAI 99. ... 


RANK = 4 WITH SIMILARITY = 0.5379004621852821 | DOC ID = 62
Open Directory - Computers: Artificial Intelligence: Machine ... 
 ... David W. Aha: Machine Learning Page - Comprehensive machine learning
 resources from Applications to Tutorials. Machine Learning ... 


RANK = 5 WITH SIMILARITY = 0.4856830826565509 | DOC ID = 81
Oxford University Machine Learning Group
 Machine Learning at the Computing Laboratory. ... Logic Programming and
 Learning

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [26]:
import nltk

nltk.download()

from nltk.corpus import wordnet as wn

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [27]:
wn.synsets('machine')

[Synset('machine.n.01'),
 Synset('machine.n.02'),
 Synset('machine.n.03'),
 Synset('machine.n.04'),
 Synset('machine.n.05'),
 Synset('car.n.01'),
 Synset('machine.v.01'),
 Synset('machine.v.02')]

2.3.3) Display all definitions (.definition()) for synsets (machine)

In [33]:
#TODO
for x in wn.synsets('machine'):
    print(x, x.definition())

Synset('machine.n.01') any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks
Synset('machine.n.02') an efficient person
Synset('machine.n.03') an intricate organization that accomplishes its goals efficiently
Synset('machine.n.04') a device for overcoming resistance at one point by applying force at some other point
Synset('machine.n.05') a group that controls the activities of a political party
Synset('car.n.01') a motor vehicle with four wheels; usually propelled by an internal combustion engine
Synset('machine.v.01') turn, shape, mold, or otherwise finish by machinery
Synset('machine.v.02') make by machinery


2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [67]:
#TODO
for x in wn.synsets('machine'):
    print(x, x.hypernyms())

Synset('machine.n.01') [Synset('device.n.01')]
Synset('machine.n.02') [Synset('person.n.01')]
Synset('machine.n.03') [Synset('organization.n.01')]
Synset('machine.n.04') [Synset('mechanical_device.n.01')]
Synset('machine.n.05') [Synset('organization.n.01')]
Synset('car.n.01') [Synset('motor_vehicle.n.01')]
Synset('machine.v.01') [Synset('shape.v.02')]
Synset('machine.v.02') [Synset('produce.v.02')]


See: http://www.nltk.org/howto/wordnet.html
for more examples